# Notebook for trying different compression options

In [17]:
import zlib
import tifffile
import time
import pickle
import scipy.signal
import numpy as np
import sys

ZLIB_COMPRESSION_LEVEL = -1

In [2]:
file_path = "/home/clewis/repos/holo-nbs/rb26_20240111/raw_voltage_chunk.tif"
data = tifffile.memmap(file_path)
data.shape

(384, 3000001)

In [3]:
# define filter functions
def butter(cutoff, fs, order=5, btype='high'):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = scipy.signal.butter(order, normal_cutoff, btype=btype, analog=False)
    return b, a


def butter_filter(data, cutoff, fs, order=5, axis=-1, btype='high'):
    b, a = butter(cutoff, fs, order=order, btype=btype)
    y = scipy.signal.filtfilt(b, a, data, axis=axis)
    return y

In [21]:
d = data[:, :150]
d = butter_filter(d, 1000, 30_000)

In [22]:
sys.getsizeof(d) / (2**30)

1.1920928955078125e-07

# Using `zlib`

### Compressing

In [10]:
t = time.time()
dz = zlib.compress(
            pickle.dumps(d, protocol=5), level=ZLIB_COMPRESSION_LEVEL
        )
print(time.time() - t)

0.008260011672973633


In [19]:
sys.getsizeof(dz) / (2**30)

0.0004132091999053955

### Decompressing

In [11]:
t = time.time()
pickle.loads(zlib.decompress(dz))
print(time.time() - t)

0.0040988922119140625


# Using straight `numpy`

### Compressing

In [24]:
t = time.time()
db = d.tobytes()
print(time.time() - t)

AttributeError: 'bytes' object has no attribute 'tobytes'

In [23]:
sys.getsizeof(db) / (2**30)

0.0004291841760277748

### Decompressing

In [27]:
t = time.time()
a = np.frombuffer(db).reshape(384, 150)
print(time.time() - t)

9.059906005859375e-05


In [26]:
a.shape

(57600,)